# Project overview

### Objective

As famous as it is for gun violence, the US still has a lack of large and easily-accessible amounts of detailed data on this topic. This project aims to change that. By creating a well-rounded dataset that entails relevant fields such as shooting incidents' location, types of guns used, shooters and victims' age and relationship, criminal landscape and demographic details of the areas where shootings occur, we will make it easier for researchers and law enforcement entities to study gun violence patterns, build predictive models for future incidents and make more informed decisions on preventive measures. 

### Datasets and tools used 
1. Gun violence dataset from [Kaggle](https://www.kaggle.com/jameslko/gun-violence-data) detailing about 240,000 gun incidents in the US in the period of 2013 through 2018.

2. US income per capita by cities and counties from [Wikipedia](https://en.wikipedia.org/wiki/List_of_United_States_counties_by_per_capita_income)

3. US crime rate by cities from [Wikipedia](https://en.wikipedia.org/wiki/List_of_United_States_cities_by_crime_rate)

4. Google Maps' [Geocoding API](https://developers.google.com/maps/documentation/geocoding/intro#Results)

5. BeautifulSoup library for scraping HTML content 

 Since it would take a lot of time to geocode 240,000 addresses, for this project we used a random sample of 2000 rows to speed up the process while maintaining the original time and geographical distribution of incidents and making sure that we would not hit Google's API limit and get charged for every API call. 

The final dataset will include: 
       `location`, `latitude`, `longitude`, `incident_id`, `date`, `state`,
       `city_or_county`, `address`, `n_killed`, `n_injured`,
       `congressional_district`, `gun_stolen`, `gun_type`,
       `incident_characteristics`, `location_description`, `n_guns_involved`,
       `notes`, `participant_age`, `participant_age_group`,
       `participant_gender`, `participant_name`, `participant_relationship`,
       `participant_status`, `participant_type`, `state_house_district`,
       `state_senate_district`, `vc_total`, `vc_murder`, `vc_rape`,
       `vc_robbery`, `vc_aggravatedAssault`, `pc_total`, `pc_burglary`,
       `pc_larcenyTheft`, `pc_motorVehicleTheft`, `arson`, `rank`,
       `per capita income`, `median household income`, `median family income`,
       `population`, `number of households`

# 1. Geocoding address data using Google's Geocoding API

### 1.1. Pulling data

In [1]:
import pandas as pd
import requests
import logging
import time

In [2]:
# Load gun violence data collected from Kaggle
full = pd.read_csv('gun-violence.csv')

In [3]:
# Generate a random sample of 2000 rows
data = full.sample(n = 2000)
data = data.dropna(subset=['address', 'city_or_county', 'state']) # remove rows with missing address
data.head()

,incident_id,date,state,city_or_county,address,n_killed,n_injured,incident_url,source_url,incident_url_fields_missing,...,participant_age,participant_age_group,participant_gender,participant_name,participant_relationship,participant_status,participant_type,sources,state_house_district,state_senate_district
190177,861097,2017-05-29,Wyoming,Casper,1500 block of S Robertson Rd,0,0,http://www.gunviolencearchive.org/incident/861097,http://trib.com/lifestyles/announcements/commu...,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://trib.com/lifestyles/announcements/commu...,59.0,29.0
27476,162831,2014-07-21,Ohio,Toledo,Ottawa Drive and Auburn Avenue,0,1,http://www.gunviolencearchive.org/incident/162831,http://www.toledoblade.com/Police-Fire/2014/07...,False,...,NaN,0::Adult 18+,0::Male,NaN,NaN,0::Injured,0::Victim,http://www.toledoblade.com/Police-Fire/2014/07...,44.0,11.0
75595,360408,2015-06-17,New York,Utica,1500 block of Howard Avenue,0,0,http://www.gunviolencearchive.org/incident/360408,http://www.uticaod.com/article/20150617/NEWS/1...,False,...,NaN,0::Adult 18+||1::Adult 18+,0::Male||1::Male,NaN,NaN,"0::Unharmed, Arrested||1::Arrested",0::Subject-Suspect||1::Subject-Suspect,http://www.uticaod.com/article/20150617/NEWS/1...,119.0,47.0
185442,837002,2017-05-04,Connecticut,Hamden,Dix Street,0,0,http://www.gunviolencearchive.org/incident/837002,http://wtnh.com/2017/05/05/hamden-police-inves...,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://wtnh.com/2017/05/05/hamden-police-inves...,94.0,17.0
128182,570537,2016-05-28,Oklahoma,Tulsa,4104 S. 130th E. Avenue,0,1,http://www.gunviolencearchive.org/incident/570537,http://www.tulsaworld.com/homepagelatest/man-s...,False,...,NaN,0::Adult 18+,0::Male,NaN,NaN,0::Injured,0::Victim,http://www.tulsaworld.com/homepagelatest/man-s...,75.0,36.0


In [4]:
# Drop unwanted columns and save the random sample generated
data = data.drop(columns = ['incident_url', 'source_url', 'incident_url_fields_missing', 'sources'])
data.to_csv('data_file.csv', index = False)

In [5]:
logger = logging.getLogger("root")
logger.setLevel(logging.DEBUG)
# create console handler
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)
logger.addHandler(ch)

In [6]:
# Set Google's Geocoding API key 
api_key = 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'

# Backoff time sets how many minutes to wait between google pings when API limit is hit
backoff_time = 30


# Set input file 
input_filename = "gun-violence.csv"

# Set output file 
output_filename = 'gun_locations.csv'

# Specify column with address data
address_column_name = "address"

# Return Full Google Results? If True, full JSON results from Google are included in output
RETURN_FULL_RESULTS = False

In [7]:
# Create a list of addresses from 3 geo columns 
addresses = (data['address'] + ', ' + data['city_or_county'] + ', ' + data['state']).tolist()
addresses[:5]

['1500 block of S Robertson Rd, Casper, Wyoming',
 'Ottawa Drive and Auburn Avenue, Toledo, Ohio',
 '1500 block of Howard Avenue, Utica, New York',
 'Dix Street, Hamden, Connecticut',
 '4104 S. 130th E. Avenue, Tulsa, Oklahoma']

Here we write a function to merge data from Google Maps' Geocoding API. 

In [8]:
def get_google_results(address, api_key=None, return_full_response=False):
    
    # Set up your Geocoding url
    geocode_url = "https://maps.googleapis.com/maps/api/geocode/json?address={}".format(address)
    if api_key is not None:
        geocode_url = geocode_url + "&key={}".format(api_key)
        
    # Ping google for the reuslts:
    results = requests.get(geocode_url)
    
    # Results will be in JSON format - convert to dict using requests functionality
    results = results.json()
    
    # if there's no results or an error, return empty results.
    if len(results['results']) == 0:
        output = {
            "formatted_address": None,
            "latitude": None,
            "longitude": None,
            "accuracy": None,
            "google_place_id": None,
            "type": None,
            "postcode": None
        }
    else:    
        answer = results['results'][0]
        output = {
            "formatted_address" : answer.get('formatted_address'),
            "latitude": answer.get('geometry').get('location').get('lat'),
            "longitude": answer.get('geometry').get('location').get('lng'),
            "zipcode": ",".join([x['long_name'] for x in answer.get('address_components') 
                                  if 'postal_code' in x.get('types')])
        }
        
    # Append some other details:    
    output['address'] = address
    output['number_of_results'] = len(results['results'])
    output['status'] = results.get('status')

    
    if return_full_response is True:
        output['response'] = results
    
    return output

Write a processing loop to parse and sort the data acquired, and debug along the way!

In [9]:
# Create a list to hold results
results = []

# Go through each address in turn
for address in addresses:
    # While the address geocoding is not finished:
    geocoded = False
    while geocoded is not True:
        # Geocode the address with google
        try:
            geocode_result = get_google_results(address, api_key, return_full_response=RETURN_FULL_RESULTS)
        except Exception as e:
            logger.exception(e)
            logger.error("Major error with {}".format(address))
            logger.error("Skipping!")
            geocoded = True
            
        # If we're over the API limit, backoff for a while and try again later.
        if geocode_result['status'] == 'OVER_QUERY_LIMIT':
            logger.info("Hit Query Limit! Backing off for a bit.")
            time.sleep(BACKOFF_TIME * 60) # sleep for 30 minutes
            geocoded = False
        else:
            # If we're ok with API use, save the results
            # Note that the results might be empty / non-ok - log this
            if geocode_result['status'] != 'OK':
                logger.warning("Error geocoding {}: {}".format(address, geocode_result['status']))
            #logger.debug("Geocoded: {}: {}".format(address, geocode_result['status']))
            results.append(geocode_result)           
            geocoded = True

    # Print status every 100 addresses
    if len(results) % 100 == 0:
        logger.info("Completed {} of {} address".format(len(results), len(addresses)))
            
    # Every 500 addresses, save progress to file(in case of a failure so we have something)
    if len(results) % 500 == 0:
        pd.DataFrame(results).to_csv("{}_bak".format(output_filename))

# All done
logger.info("Finished geocoding all addresses")

# Write the full results to csv 
pd.DataFrame(results).to_csv(output_filename, encoding='utf8')

Completed 100 of 1873 address
Completed 200 of 1873 address
Completed 300 of 1873 address
Completed 400 of 1873 address
Completed 500 of 1873 address
Completed 600 of 1873 address
Completed 700 of 1873 address
Completed 800 of 1873 address
Completed 900 of 1873 address
Completed 1000 of 1873 address
Completed 1100 of 1873 address
Error geocoding 961 NJ-10 #2F, Randolph, New Jersey: REQUEST_DENIED
Completed 1200 of 1873 address
Completed 1300 of 1873 address
Completed 1400 of 1873 address
Completed 1500 of 1873 address
Completed 1600 of 1873 address
Completed 1700 of 1873 address
Completed 1800 of 1873 address
Finished geocoding all addresses


### 1.2. Integrating latitude and longitude data to the gun violence dataset

Now that we already got the latitude and longitude of the all shooting incidents, we will be merging it with the original dataset by matching `location`.

In [10]:
# Load data
geodf = pd.read_csv('gun_locations.csv')

# Select columns of interest
geodf = geodf[['address', 'latitude', 'longitude']]

# Change column name
geodf.columns = ['location', 'latitude', 'longitude']
geodf.head()

,location,latitude,longitude
0,"1500 block of S Robertson Rd, Casper, Wyoming",42.828600,-106.415417
1,"Ottawa Drive and Auburn Avenue, Toledo, Ohio",41.668732,-83.579195
2,"1500 block of Howard Avenue, Utica, New York",43.089293,-75.236325
3,"Dix Street, Hamden, Connecticut",41.340413,-72.938212
4,"4104 S. 130th E. Avenue, Tulsa, Oklahoma",36.104019,-95.832666


In [11]:
# Create an address column that combines all geographical data 
data = data.assign(location = lambda x: data.address + ', ' + data.city_or_county + ', ' + data.state)
data['location'].head()

190177    1500 block of S Robertson Rd, Casper, Wyoming
27476      Ottawa Drive and Auburn Avenue, Toledo, Ohio
75595      1500 block of Howard Avenue, Utica, New York
185442                  Dix Street, Hamden, Connecticut
128182         4104 S. 130th E. Avenue, Tulsa, Oklahoma
Name: location, dtype: object

In [12]:
# Merge data
geodata = pd.merge(geodf, data, on = 'location', how = 'inner')
geodata = geodata.sort_values(['date'], axis = 0, ascending = True) # Sort data in time order
geodata.head()

,location,latitude,longitude,incident_id,date,state,city_or_county,address,n_killed,n_injured,...,notes,participant_age,participant_age_group,participant_gender,participant_name,participant_relationship,participant_status,participant_type,state_house_district,state_senate_district
173,"Congress Street, Lynn, Massachusetts",42.467899,-70.961057,95620,2014-01-01,Massachusetts,Lynn,Congress Street,0,1,...,NaN,NaN,0::Adult 18+,0::Male,NaN,NaN,0::Injured,0::Victim,NaN,NaN
436,"3200 block of West Highland Street, Springfiel...",37.144570,-93.344794,95255,2014-01-01,Missouri,Springfield,3200 block of West Highland Street,0,0,...,NaN,0::33||1::35,0::Adult 18+||1::Adult 18+,0::Male||1::Male,0::Marcus Andrews||1::Iver L. Anthony,NaN,0::Unharmed||1::Unharmed,0::Subject-Suspect||1::Subject-Suspect,133.0,30.0
1390,"Myrtis Street, New Iberia, Louisiana",30.006505,-91.839744,93155,2014-01-04,Louisiana,New Iberia,Myrtis Street,0,1,...,NaN,0::23||1::21||2::19||3::21||4::20,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male||1::Male||2::Male||3::Male||4::Male,1::David Arceneaux||2::Jabrisin Kinchen||3::Ja...,NaN,0::Injured||1::Unharmed||2::Unharmed||3::Unhar...,0::Victim||1::Subject-Suspect||2::Subject-Susp...,96.0,22.0
1561,"900 block of Fern Ridge Road, Virginia Beach, ...",36.807460,-76.083657,94800,2014-01-06,Virginia,Virginia Beach,900 block of Fern Ridge Road,0,0,...,all perps described as beng in mid 20s,NaN,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,2::Male||3::Male||4::Male,NaN,NaN,0::Unharmed||1::Unharmed||2::Unharmed||3::Unha...,0::Victim||1::Victim||2::Subject-Suspect||3::S...,84.0,8.0
243,"Curry Street, Columbus, Georgia",32.452363,-84.922394,93647,2014-01-06,Georgia,Columbus,Curry Street,0,0,...,NaN,0::42||1::42||2::11,0::Adult 18+||1::Adult 18+||2::Child 0-11||3::...,NaN,NaN,NaN,0::Unharmed||1::Unharmed||2::Unharmed||3::Unha...,0::Victim||1::Victim||2::Victim||3::Victim,136.0,15.0


In [13]:
# Save the dataframe
geodata.to_csv('geodata.csv', index = False)

# 2. Scraping demographic data using BeautifulSoup

In [14]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import csv

To extract the information from the wiki pages on crime rate and per capita income, BeautifulSoup is used to scrape the webpage. Then, headers, rows, and columns are obtained and the information is written into a csv outfile. This allows us to get all the information into csv format which can then be merged to obtain meaningful information about gun violence and the locations they occurred. 

This program takes a wiki page link and parses it using BeautifulSoup.
First, the table from the page is extracted from the soup using the table tag: "'table',{'class': 'wikitable sortable'}". Then the headers are obtained with the header tag ('th'). Superscripts are removed from the soup by finding them with the 'sup' tag and using 'replaceWith' to overwrite the tag and its content. Next, the rows are separated using the Beautiful Soup tag 'td'. Finally, all of these components are written into a new csv file using 'csv.writer'. 

### 2.1. Crime data

In [15]:
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_United_States_cities_by_crime_rate').text
soup = BeautifulSoup(website_url,'lxml')
#print(soup.prettify())

Look for table tags in the html using BeautifulSoup

In [16]:
my_table = soup.find('table',{'class': 'wikitable sortable'})
#my_table

Headers are extracted using the html header tag

In [17]:
temp_headers = [header.text for header in my_table.find_all('th')]
for header in temp_headers:
    header = header.strip()
    print(header)

State
City
Population
Violent crime
Property crime
Arson2
Total
Murder and
Nonnegligent manslaughter
Rape1
Robbery
Aggravated assault
Total
Burglary
Larceny-theft
Motor vehicle theft


In [18]:
headers = ['state', 'city', 'population','vc_total', 'vc_murder', 'vc_rape', 'vc_robbery', 'vc_aggravatedAssault','pc_total', 'pc_burglary', 'pc_larcenyTheft', 'pc_motorVehicleTheft', 'arson']

Superscripts are removed from the text using the 'sup' tag. 

It was a challenge to remove the superscript because the most common approach is to delete the tag but keep the content but I had to find a way ('replaceWith') which removed the tag along with its content. 

In [19]:
for tag in my_table.find_all('sup'):
    tag.replaceWith('')
#print(my_table)

In [20]:
rows = []
for row in my_table.find_all('tr'):    
    rows.append([val.text.strip() for val in row.find_all('td')])

Transfer the information (headers and rows) into a csv file

Initially when I was writing into the csv it created empty rows between each row and it was challenge to find a way to remove the empty rows. I found that with Python3 you have to add "newline = ''" to remove the empty rows.

In [21]:
with open('crime_data.csv', 'w', newline ='') as f:
    writer = csv.writer(f)
    writer.writerow(headers)
    writer.writerows(row for row in rows if row)

In [22]:
crime = pd.read_csv('crime_data.csv')
crime.head()

,state,city,population,vc_total,vc_murder,vc_rape,vc_robbery,vc_aggravatedAssault,pc_total,pc_burglary,pc_larcenyTheft,pc_motorVehicleTheft,arson
0,Alabama,Mobile,248431,740.25,20.13,57.16,177.11,485.85,5453.83,1216.84,3730.21,506.78,22.94
1,Alaska,Anchorage,296188,1203.29,9.12,132.01,262.67,799.49,5415.82,748.17,3619.66,1047.98,20.93
2,Arizona,Chandler,249355,259.47,2.01,52.13,56.95,148.38,2329.61,314.41,1866.01,149.18,NaN
3,Arizona,Gilbert,242090,85.51,2.07,16.11,21.07,46.26,1385.85,192.49,1137.59,55.76,12.39
4,Arizona,Glendale,249273,488.22,4.81,38.91,192.96,251.53,4530.37,637.45,3426.36,466.56,19.26


### 2.2. Income per capita

In [23]:
wiki_url = requests.get('https://en.wikipedia.org/wiki/List_of_United_States_counties_by_per_capita_income').text
wiki_soup = BeautifulSoup(wiki_url,'lxml')
#print(wiki_soup.prettify())

In [24]:
wiki_table = wiki_soup.find('table',{'class': 'wikitable sortable'})
#wiki_table

It was a challenge to get the headings from the wiki table to line up with the specific data in the csv, so I hardcoded the headers into the file. 

In [25]:
# Create a list of headers
income_headers = ['rank','county', 'state','per capita income','median household income','median family income', 'population', 'number of households']

In [26]:
all_rows = []
for row in wiki_table.find_all('tr'):
    all_rows.append([val.text.strip() for val in row.find_all('td')])

In [27]:
#all_rows

In [28]:
with open('income_data.csv', 'w', newline = '') as file:
    wiki_writer = csv.writer(file)
    wiki_writer.writerow(income_headers)
    wiki_writer.writerows(row for row in all_rows if row)

In [29]:
income = pd.read_csv('income_data.csv', encoding = 'ISO-8859-1')
income.head()

,rank,county,state,per capita income,median household income,median family income,population,number of households
0,1,New York County,New York,"$62,498","$69,659","$84,627","1,605,272","736,192"
1,2,Arlington,Virginia,"$62,018","$103,208","$139,244","214,861","94,454"
2,3,Falls Church City,Virginia,"$59,088","$120,000","$152,857","12,731","5,020"
3,4,Marin,California,"$56,791","$90,839","$117,357","254,643","102,912"
4,5,Alexandria City,Virginia,"$54,608","$85,706","$107,511","143,684","65,369"


# 3. Integrate demographic data to geographical data

In [31]:
geo = pd.read_csv('geodata.csv')
crime = pd.read_csv('crime_data.csv')
income = pd.read_csv('income_data.csv', encoding = 'ISO-8859-1')

Since geographical columns are named differently in each dataset, we need to change these columns' name and drop columns that will be duplicated (like `population`, which exists in both the `crime` and `income` datasets.  

In [32]:
# Change column names and drop unwanted columns
income = income.rename(columns = {'county':'city_or_county'}) 
crime = crime.rename(columns = {'city':'city_or_county'})

# Drop the population column
crime = crime.drop(columns = ['population'])

# Check all columns
print(income.columns)
print(crime.columns)

Index(['rank', 'city_or_county', 'state', 'per capita income',
       'median household income', 'median family income', 'population',
       'number of households'],
      dtype='object')
Index(['state', 'city_or_county', 'vc_total', 'vc_murder', 'vc_rape',
       'vc_robbery', 'vc_aggravatedAssault', 'pc_total', 'pc_burglary',
       'pc_larcenyTheft', 'pc_motorVehicleTheft', 'arson'],
      dtype='object')


We are going to merge the 3 dataframes into a single one on `state` and `city_or_county` columns. 

In [33]:
# Merge the gun violence data with crime data 
final_data = pd.merge(geo, crime, on = ['state', 'city_or_county'])

# Merge the dataset just created with income data
final_data = pd.merge(final_data, income, on = ['state', 'city_or_county'])
final_data.head()

,location,latitude,longitude,incident_id,date,state,city_or_county,address,n_killed,n_injured,...,pc_burglary,pc_larcenyTheft,pc_motorVehicleTheft,arson,rank,per capita income,median household income,median family income,population,number of households
0,"Fernwood in University Oaks, Houston, Texas",29.706851,-95.350625,94695,2014-01-09,Texas,Houston,Fernwood in University Oaks,0,0,...,731.66,2900.82,495.93,28.44,2550,"$19,090","$33,119","$46,557","23,365","7,889"
1,"Valero on North Shepherd at 16th Street, Houst...",29.799452,-95.409716,97433,2014-01-15,Texas,Houston,Valero on North Shepherd at 16th Street,0,0,...,731.66,2900.82,495.93,28.44,2550,"$19,090","$33,119","$46,557","23,365","7,889"
2,"Bush Intercontinental Airport, Houston, Texas",29.990220,-95.336783,109526,2014-01-27,Texas,Houston,Bush Intercontinental Airport,0,0,...,731.66,2900.82,495.93,28.44,2550,"$19,090","$33,119","$46,557","23,365","7,889"
3,"Hobby Airport, Houston, Texas",29.645908,-95.276905,113315,2014-03-03,Texas,Houston,Hobby Airport,0,0,...,731.66,2900.82,495.93,28.44,2550,"$19,090","$33,119","$46,557","23,365","7,889"
4,"300 block of Airtex Blvd., Houston, Texas",29.982697,-95.428977,202050,2014-10-06,Texas,Houston,300 block of Airtex Blvd.,0,1,...,731.66,2900.82,495.93,28.44,2550,"$19,090","$33,119","$46,557","23,365","7,889"


In [34]:
# Check all columns again 
final_data.columns

Index(['location', 'latitude', 'longitude', 'incident_id', 'date', 'state',
       'city_or_county', 'address', 'n_killed', 'n_injured',
       'congressional_district', 'gun_stolen', 'gun_type',
       'incident_characteristics', 'location_description', 'n_guns_involved',
       'notes', 'participant_age', 'participant_age_group',
       'participant_gender', 'participant_name', 'participant_relationship',
       'participant_status', 'participant_type', 'state_house_district',
       'state_senate_district', 'vc_total', 'vc_murder', 'vc_rape',
       'vc_robbery', 'vc_aggravatedAssault', 'pc_total', 'pc_burglary',
       'pc_larcenyTheft', 'pc_motorVehicleTheft', 'arson', 'rank',
       'per capita income', 'median household income', 'median family income',
       'population', 'number of households'],
      dtype='object')

In [36]:
final_data.to_csv("final_data.csv", index = False)

# Conclusion 
The features created can be used to plot shooting incidents onto a US map to see the hottest areas of gun violence and many other interesting trends. The project was implemented within a short amount of time so there might be many great data sources that hadn't been explored. Further improvement could be made by building a larger dataset (i.e. more observations) and adding gun control policies in each state to assist analyzing the impact of gun policies on shooting patterns.  